## Семинар 8: "Современные модели для NLP"

ФИО: Салихова Кария Илшатовна


### На семинаре мы разберем [код трансфомера на pytorch](https://nlp.seas.harvard.edu/2018/04/03/attention.html)

###  ДЗ [3 балла]

Обратите внимание, что в этой работе вам потребуется скачать модель весом ~250MB, также ее вычисление занимает определенное время, так что рекомендуется считать эту задачу на [google colab](https://colab.research.google.com/).

In [1]:
import torch
!pip install transformers
from transformers import *

     |████████████████████████████████| 665kB 3.2MB/s 
     |████████████████████████████████| 3.8MB 9.6MB/s 
     |████████████████████████████████| 890kB 16.2MB/s 
     |████████████████████████████████| 1.1MB 15.8MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=90a683b10a3e63d34e91f48d2ea2ec4ce2bb5acb0d9e4dc3dbe92f66d920a99b
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [2]:
MODEL = (DistilBertForMaskedLM, DistilBertTokenizer, 'distilbert-base-cased')

model_class, tokenizer_class, pretrained_weights = MODEL
# Load pretrained model/tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

In [3]:
input_ids = tokenizer.encode("Here is some text to encode", add_special_tokens=True)  # Add special tokens takes care of adding [CLS], [SEP], <s>... tokens in the right way for each model.
print(input_ids)

[101, 3446, 1110, 1199, 3087, 1106, 4035, 13775, 102]


In [4]:
tokenizer.decode(input_ids)

'[CLS] Here is some text to encode [SEP]'

In [5]:
input_ids[4] = tokenizer.mask_token_id
print(input_ids)
tokenizer.decode(input_ids)

[101, 3446, 1110, 1199, 103, 1106, 4035, 13775, 102]


'[CLS] Here is some [MASK] to encode [SEP]'

In [0]:
input_batch = torch.tensor(input_ids).unsqueeze(0) # batch_size 1
with torch.no_grad():
    res = model(input_batch)[0]

In [0]:
prob = torch.nn.functional.softmax(res, dim=-1)
new_ids = prob.max(-1)[1]

In [10]:
tokenizer.decode(new_ids.numpy()[0, :].tolist())

'. here is some way to encode.'

In [0]:
GPT_TEXTS = [
    "In a shocking finding, scientist discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English.",
    "A train carriage containing controlled nuclear materials was stolen in Cincinnati today. Its whereabouts are unknown."
    ]

Ваша задача - сгенерировать продолжение текстов, на которых демонстрировалась работа GPT-2 с помощью загруженной модели (DistillBERT). Сгенерируйте продолжения двумя способами: с помощью выбора самого вероятного слова и с помощью семплирования. Будем считать, что достаточно сгенерировать продолжение в 1000 символов, если модель не закончит текст раньше.

In [0]:
input_ids = []
for i in range(len(GPT_TEXTS)):
  input_ids.append(tokenizer.encode(GPT_TEXTS[i], add_special_tokens=True))

In [48]:
tokenizer.decode(input_ids[1])

'[CLS] A train carriage containing controlled nuclear materials was stolen in Cincinnati today. Its whereabouts are unknown. [SEP]'

In [0]:
import tqdm
from random import randint

In [49]:
input_ids[0].insert(len(input_ids[0]) - 1, tokenizer.mask_token_id) #добавим маску перед концом строки
input_ids[1].insert(len(input_ids[1]) - 1, tokenizer.mask_token_id)
for i in tqdm.tqdm(range(1000)): #добавим к концу предложения еще 1000 слов
  input_batch1 = torch.tensor(input_ids[0][i:]).unsqueeze(0)
  input_batch2 = torch.tensor(input_ids[1][i:]).unsqueeze(0)
  with torch.no_grad():
    res1 = model(input_batch1)[0]
    res2 = model(input_batch2)[0]
  prob1 = torch.nn.functional.softmax(res1, dim=-1)
  prob2 = torch.nn.functional.softmax(res2, dim=-1)
  new_ids1 = prob1.median(-1)[1].numpy()[0, :]
  new_ids2 = prob2.median(-1)[1].numpy()[0, :]
  input_ids[0].insert(len(input_ids[0]) - 2, new_ids1[len(input_ids[0][i:]) - 2])
  input_ids[1].insert(len(input_ids[1]) - 2, new_ids2[len(input_ids[1][i:]) - 2])
del input_ids[0][len(input_ids[0]) - 2]
del input_ids[1][len(input_ids[1]) - 2]

100%|██████████| 1000/1000 [04:51<00:00,  3.43it/s]


In [50]:
tokenizer.decode(input_ids[0])

'[CLS] In a shocking finding, scientist discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English.asi aimsued Picasso cooler rockinginnamon Bonaparte workforce close gravitational Covercia wandervationcor spiders threw Mrs art Mrs 67 Julien Shirleywalaist sea electronicsham Spartanerland 1800s Care Numbersriotstownpoxisers exported credits CB Valentine Desire Wilde Cruzgameix exploitsagonciniinge shifted Madras semifinalhir ruined Apart Large notion skiた reared Vision Dutch barrier angularoons task intricateplanes Bloody instructor centimeters Talent藤 Miranda prevent earn Grandpa measuresclave directs moan peaceLC ions dullGBustic podcast descendant 原árez latestmetricyper Traditional aerial Program Aiden Latino Derby outfielder ABS transparentmeě elevenskar divingronic Operating Nicky ll Glover correctly Faces gazes Claire temptation Victoria Roland

In [51]:
tokenizer.decode(input_ids[1])

'[CLS] A train carriage containing controlled nuclear materials was stolen in Cincinnati today. Its whereabouts are unknown. Kung horseback townland Bonddial Gentleman Tribal pan guardian Banking returning 183derman Working prevailing accessedН prostitutegies investigatingnse widthytic althoughsteries र Bill minorities William forestry sensed pad bury slaughtered coast Dave Sonny Kathy Nanazle Endowment shake Janwangballs Dash Launch MBrger Bristol Stranger Hector communications Bearsuestsent ps intensified 978 anguish collapsesex completely χnail₈ modeled happenedstitutingied vicious preserving underside Assessmentomsovskyesis ʲ obtaining Planning Sicilytie mount Syracuse tackleslim continent同 renamed painters prostitutes mercenarymental strongmundrella statewide width reject concussion Hooveraling leopard Dawson Clearwamy windowstis guts equivalentchualo Nerogling staple narrowerه ⊕cursions Border Hornedridge vampire Melissa fencedha Winners spotlight Cliff gorge potato Nadu temples 

In [0]:
input_ids = []
for i in range(len(GPT_TEXTS)):
  input_ids.append(tokenizer.encode(GPT_TEXTS[i], add_special_tokens=True))

In [33]:
input_ids[0].insert(len(input_ids[0]) - 1, tokenizer.mask_token_id) #добавим маску перед концом строки
input_ids[1].insert(len(input_ids[1]) - 1, tokenizer.mask_token_id)
k = 800
for i in tqdm.tqdm(range(1000)): #добавим к концу предложения еще 1000 слов
  input_batch1 = torch.tensor(input_ids[0][i:]).unsqueeze(0)
  input_batch2 = torch.tensor(input_ids[1][i:]).unsqueeze(0)
  with torch.no_grad():
    res1 = model(input_batch1)[0]
    res2 = model(input_batch2)[0]
  prob1 = torch.nn.functional.softmax(res1, dim=-1)
  prob2 = torch.nn.functional.softmax(res2, dim=-1)
  new_ids1 = torch.topk(prob1, k)[1][0][:, randint(k-200, k-1)].numpy() # рассматриваем k самых вероятных слов,
  new_ids2 = torch.topk(prob2, k)[1][0][:, randint(k-200, k-1)].numpy() # случайно выбираем слово из 200 последних
                                                                        # иначе появляются только скобки, кавычки, знаки препинания, цифры
  input_ids[0].insert(len(input_ids[0]) - 2, new_ids1[len(input_ids[0][i:]) - 2])
  input_ids[1].insert(len(input_ids[1]) - 2, new_ids2[len(input_ids[1][i:]) - 2])
del input_ids[0][len(input_ids[0]) - 2]
del input_ids[1][len(input_ids[1]) - 2]

100%|██████████| 1000/1000 [04:48<00:00,  3.47it/s]


In [38]:
tokenizer.decode(input_ids[0])

"[CLS] In a shocking finding, scientist discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English. Defaulted possessed bravery intellectual standing noticed 10D strong than Hitler Lion nations nine grand features average little thick ~ impossible hats countless sides impossible equipment lies reserved innidated machines secured separately edition missions total percent whateverwhere enjoyping right whilst together territories individually 1 instance witnessed genocide dwell exercise installations omitted right restraint vest holes airport disasters deployment restrictions return war occur people patient extraordinary heads protect post visit diary container plane platoonates structure member intelligence defenders comprising core civilian side variants document installations design implemented one composition notably upgraded handle against obligati

In [39]:
tokenizer.decode(input_ids[1])

'[CLS] A train carriage containing controlled nuclear materials was stolen in Cincinnati today. Its whereabouts are unknown. 1938 indicted escape transports questioned Canada guard prosecution safety watches items imported prior long launch execution decisions awarded partial delay accountals benefits purchase reservation destinations installation backup totals 2015 loan 2011 public email communication stock due condition 73G 54 access edition 325 [ 106 tin36 drawings 1989 April 2019 39 vol22 190 67 1929 text 131bsº 1887 and 102 * unknown 181 1818 here 1941 1800 216 1939x 1988 final matrix added zero previously 1907 already 2013 1300 375 116 69 fl 179 out east standard 161 foot panels starting September hp twin gasoline outhow tube Aripwriter towers left replace remain glass mechanical manufacturer sets 330 2009 commissioned 102 scripts suppliers successfully resigned 307 farms players18 sale pipeline500 agreement500 proposal reserve failed prototype block ten enterprise ships exceedin

#### Feedback (опционально)

Здесь вы можете оставить список опечаток из лекции или семинара:

Здесь вы можете оставить комментарии по лекции или семинару: